In [27]:
import time
import qiskit
from qiskit import IBMQ, assemble, transpile
from qiskit.providers.jobstatus import JOB_FINAL_STATES
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise # import AER noise model
import numpy as np
import qiskit.ignis.verification.tomography as tom
from qiskit.providers.ibmq.managed import IBMQJobManager


In [28]:
# Uncomment below two lines on first use.
# IBMQ.enable_account("9dc5c937fcedf3575d64912c02715c0ec0e27043cf5f18040b0304cdd16dac2ea042ffecad4b007b245926a694f92a9ebfdc1ede6b381b004a2fe0ed2cf262b1")
# provider = IBMQ.load_account()
provider.backends()
# backend = provider.backends.ibmq_vigo



[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [31]:
qubit_list = [0,1,2,3,4]
q1 = QuantumRegister(len(qubit_list))
c1 = ClassicalRegister(len(qubit_list))

qc = QuantumCircuit(q1)
# qc2 = QuantumCircuit(q1)
# for j in range(5):
#     qc2.h(q1[j])
# qc2.cz(q1[0], q1[1])
# qc2.cz(q1[2], q1[3])
# qc2.cz(q1[1], q1[2])
# qc2.cz(q1[3], q1[4])
# qc2.swap(q1[1],q1[4])
# qc2.cz(q1[0], q1[1])
# print(qc2)


#more optimal circuit
qc.h(q1[1])
qc.h(q1[3])
qc.cx(q1[1],q1[0])
qc.cx(q1[3],q1[2])
qc.cx(q1[1],q1[2])
qc.cx(q1[3],q1[4])
qc.h(q1[2])
qc.h(q1[4])
qc.swap(q1[1],q1[4])
qc.cx(q1[1],q1[0])
qc.h(q1[0])

#anirudh pytket circuit
qc = QuantumCircuit(q1)
qc.h(q1[1])
qc.cx(q1[1],q1[0])
qc.h(q1[0])
qc.h(q1[1])
qc.cx(q1[0],q1[4])
qc.h(q1[4])
qc.cx(q1[4],q1[3])
qc.h(q1[3])
qc.cx(q1[3],q1[2])
qc.h(q1[2])
qc.swap(q1[2],q1[3])
qc.cx(q1[3],q1[1])
qc.h(q1[1])

print(qc)

bk_qasm = qiskit.Aer.get_backend('qasm_simulator')
bk_sv = qiskit.Aer.get_backend('statevector_simulator')
bkIBM_16 = provider.get_backend(name='ibmq_16_melbourne')
bkIBM_5 = provider.get_backend(name='ibmqx2')
bkIBM_qasm = provider.get_backend('ibmq_qasm_simulator')


              ┌───┐┌───┐                                           
q2434_0: ─────┤ X ├┤ H ├──■────────────────────────────────────────
         ┌───┐└─┬─┘├───┤  │                              ┌───┐┌───┐
q2434_1: ┤ H ├──■──┤ H ├──┼──────────────────────────────┤ X ├┤ H ├
         └───┘     └───┘  │                 ┌───┐┌───┐   └─┬─┘└───┘
q2434_2: ─────────────────┼─────────────────┤ X ├┤ H ├─X───┼───────
                          │       ┌───┐┌───┐└─┬─┘└───┘ │   │       
q2434_3: ─────────────────┼───────┤ X ├┤ H ├──■────────X───■───────
                        ┌─┴─┐┌───┐└─┬─┘└───┘                       
q2434_4: ───────────────┤ X ├┤ H ├──■──────────────────────────────
                        └───┘└───┘                                 


In [32]:
# state vector simulation
job = qiskit.execute(qc, bk_sv)
psi_qc = job.result().get_statevector(qc)

In [33]:
# qasm simulation
qctom = tom.state_tomography_circuits(qc, qubit_list, meas_labels='Pauli', meas_basis='Pauli')

# Execute the [[5,1,3]] state prep circuit (with the same noise model)
# job = qiskit.execute(ghztom, backend=backend, optimization_level=3, shots=1000, noise_model=noise_model)
job = qiskit.execute(qctom, backend=bk_qasm, optimization_level=3, shots=4096)
results = job.result()

tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# Perform the tomography fit
# which outputs a density matrix
rho_qc = tomo_qc.fit(method='lstsq')


# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


Fit Fidelity = 0.992566709416032


In [34]:
# running on the machine
bcknd = bkIBM_16
max_job_count = 5
max_exp_per_job = 54

# qc_transpiled1 = qc.decompose()
# qc_transpiled1 = qc_transpiled1.decompose()


# print(qc_transpiled1)
# qc_transpiled = transpile(qc_transpiled1, backend=bcknd, optimization_level=0, initial_layout=[2,3,11,10,4], backend_properties=bckprop)

# print(qc_transpiled)

# qc = circuit

qctom = tom.state_tomography_circuits(qc, qubit_list, meas_labels='Pauli', meas_basis='Pauli')
# need to add measurements of syndromes and flags here
qctom = transpile(qctom, backend=bcknd, optimization_level=0, initial_layout=[0,1,12,13,14])

print(qctom[0])
print(qctom[3**len(qubit_list)-1])

                            ┌───┐┌──────────┐                                  »
   q2434_0 -> 0 ────────────┤ X ├┤ U2(0,pi) ├──■───────────────────────────────»
                ┌──────────┐└─┬─┘├──────────┤  │                               »
   q2434_1 -> 1 ┤ U2(0,pi) ├──■──┤ U2(0,pi) ├──┼───────────────────────────────»
                └──────────┘     └──────────┘  │                               »
 ancilla_0 -> 2 ───────────────────────────────┼───────────────────────────────»
                                               │                               »
 ancilla_1 -> 3 ───────────────────────────────┼───────────────────────────────»
                                               │                               »
 ancilla_2 -> 4 ───────────────────────────────┼───────────────────────────────»
                                               │                               »
 ancilla_3 -> 5 ───────────────────────────────┼───────────────────────────────»
                            

In [87]:
print(qc_transpiled.size())
print(qc_transpiled.depth())
# bckprop = bcknd.properties()
# bckconf = bcknd.configuration()
# print(bckprop)

13
9


In [35]:
# running on machine (continued)
job_manager = IBMQJobManager()
job_set_foo = job_manager.run(qctom, backend=bcknd, name='qctom', shots=2048, max_experiments_per_job=max_exp_per_job)

# single job
# qobj = assemble(, bkIBM_5, shots=2048)
# job = bkIBM_5.run(qobj)

start_time = time.time()
job_status = job_set_foo.statuses()
while job_status[max_job_count - 1] not in JOB_FINAL_STATES:
    print(f'Status @ {time.time()-start_time:0.0f} s: {job_status[max_job_count - 1]},'
          f' ')
    time.sleep(10)
    job_status = job_set_foo.statuses()
#     print(job_status)

results = job_set_foo.results()
# for _ in results:
#     print(results.get_counts(_))
    
tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# Perform the tomography fit
# which outputs a density matrix
rho_qc = tomo_qc.fit(method='lstsq')

# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 52 s: JobStatus.QUEUED, 
Status @ 104 s: JobStatus.QUEUED, 
Status @ 156 s: JobStatus.QUEUED, 
Status @ 208 s: JobStatus.QUEUED, 
Status @ 259 s: JobStatus.QUEUED, 
Status @ 311 s: JobStatus.QUEUED, 
Status @ 363 s: JobStatus.QUEUED, 
Status @ 415 s: JobStatus.QUEUED, 
Status @ 467 s: JobStatus.QUEUED, 
Status @ 519 s: JobStatus.QUEUED, 
Status @ 571 s: JobStatus.QUEUED, 
Status @ 623 s: JobStatus.QUEUED, 
Status @ 675 s: JobStatus.QUEUED, 
Status @ 726 s: JobStatus.QUEUED, 
Status @ 777 s: JobStatus.QUEUED, 
Fit Fidelity = 0.10451004252945567


In [36]:
# results = job_set_foo.results()
for i in range(3**(len(qubit_list))):
    print(results.get_counts(i))
print(i)
# print(results.get_counts(8))
# tomo_qc = tom.StateTomographyFitter(results, qctom, meas_basis='Pauli')
# # Perform the tomography fit
# # which outputs a density matrix
# rho_qc = tomo_qc.fit(method='lstsq')

{'01111': 96, '10101': 54, '00010': 112, '11101': 27, '10100': 62, '01001': 107, '10001': 69, '11010': 105, '01100': 49, '01010': 80, '00111': 80, '11110': 23, '10110': 44, '11011': 34, '00101': 69, '01000': 52, '10011': 72, '10111': 50, '11100': 92, '01011': 49, '00110': 63, '10010': 74, '11111': 47, '11000': 40, '01101': 54, '00100': 90, '01110': 40, '11001': 42, '00001': 64, '10000': 45, '00000': 89, '00011': 74}
{'01111': 75, '10101': 48, '00010': 73, '11101': 33, '10100': 81, '01001': 85, '10001': 72, '11010': 95, '01100': 83, '01010': 87, '00111': 82, '11110': 38, '10110': 57, '11011': 33, '00101': 58, '01000': 45, '10011': 62, '10111': 45, '11100': 48, '01011': 51, '00110': 55, '10010': 105, '11111': 81, '11000': 60, '01101': 40, '00100': 71, '01110': 51, '11001': 77, '00001': 63, '10000': 60, '00000': 55, '00011': 79}
{'01111': 58, '10101': 39, '00010': 180, '11101': 38, '10100': 30, '01001': 67, '10001': 76, '11010': 20, '01100': 106, '01010': 152, '00111': 68, '11110': 31, '1

In [94]:
# Fidelity calculation

F_qc = state_fidelity(psi_qc, rho_qc)
print('Fit Fidelity =', F_qc)


Fit Fidelity = 0.3558957715110188


In [13]:
# result.data()
# result.get_memory()
# result.get_counts()
# result.get_statevector
# result.get_unitary
# print(job_set_foo.statuses)
# results = job_set_foo.results()

print(qctom[0])


                ┌──────────┐                           ┌───┐          »
      q0_2 -> 0 ┤ U2(0,pi) ├──■────────────────■───────┤ X ├───────■──»
                └──────────┘┌─┴─┐┌──────────┐  │       └─┬─┘┌───┐  │  »
      q0_3 -> 1 ────────────┤ X ├┤ U2(0,pi) ├──┼────■────┼──┤ X ├──┼──»
                ┌──────────┐└───┘└──────────┘  │  ┌─┴─┐  │  └─┬─┘  │  »
      q0_4 -> 2 ┤ U2(0,pi) ├───────────────────┼──┤ X ├──┼────■────┼──»
                └──────────┘                   │  └───┘  │         │  »
 ancilla_0 -> 3 ───────────────────────────────┼─────────┼─────────┼──»
                                               │         │         │  »
 ancilla_1 -> 4 ───────────────────────────────┼─────────┼─────────┼──»
                                               │         │         │  »
 ancilla_2 -> 5 ───────────────────────────────┼─────────┼─────────┼──»
                                               │         │         │  »
 ancilla_3 -> 6 ───────────────────────────────┼─────────┼──────

In [18]:
import pickle

def unpickling_data(filename):
    file = open(filename,'rb')
    new_data = pickle.load(file)
    file.close()
    return new_data

In [19]:
circuit = unpickling_data('qc1.p')
circuit.draw(output='mpl')


AttributeError: _name